In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import os
from kaggle_secrets import UserSecretsClient
os.environ["OPENAI_API_KEY"] = UserSecretsClient().get_secret("OPENAI_API_KEY")

In [3]:
!pip install -q youtube-transcript-api langchain-community langchain-openai \
               faiss-cpu tiktoken python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.1/485.1 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.6/84.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.1/476.1 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 77.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 343.7/343.7 kB 15.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xmanager 0.7.1 requires sqlalchemy==1.2.19, but you have sqlalchemy 2.0.45 which is incompatible.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.2.2 which

In [4]:
from youtube_transcript_api import YouTubeTranscriptApi
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate

2025-12-17 17:42:34.683273: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765993354.945250      17 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765993355.046159      17 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765993355.697674      17 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765993355.697731      17 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765993355.697734      17 computation_placer.cc:177] computation placer alr

In [5]:
from youtube_transcript_api import YouTubeTranscriptApi

In [6]:
pip install youtube-transcript-api

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install --upgrade youtube-transcript-api bB

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.7 MB/s eta 0:00:00
  Created wheel for bB: filename=bb-0.1-py3-none-any.whl size=2208 sha256=a35836b83b8a4bb7f76cc92e00685e7a9f0cf4176704c386a8bf86101aad3bbb
  Stored in directory: /root/.cache/pip/wheels/b2/fb/8d/49b78dc720714304eb21e6f19bd9516740e84ca754b72b2009
  Created wheel for bitbucket-api: filename=bitbucket_api-0.5.0-py3-none-any.whl size=13743 sha256=a796ad9ccc5889442a093287bcd5b4fcbcc70606fa40b4e49f158b662fa85918
  Stored in directory: /root/.cache/pip/wheels/e0/6c/33/d72389a50d42850c2448b8beb85b11acd45f9b89d0870e57f9
Successfully built bB bitbucket-api
Note: you may need to restart the kernel to use updated packages.


In [8]:
from youtube_transcript_api import YouTubeTranscriptApi
video_id = "ockrdZal5fA"
ytt_api = YouTubeTranscriptApi()
transcript_a = ytt_api.fetch(video_id, languages=['en'])

In [9]:
transcript_a

FetchedTranscript(snippets=[FetchedTranscriptSnippet(text="So what's going to be the future of", start=5.52, duration=4.159), FetchedTranscriptSnippet(text='software engineers? Are they at threat?', start=6.879, duration=4.241), FetchedTranscriptSnippet(text='Uh', start=9.679, duration=2.721), FetchedTranscriptSnippet(text=">> Shimos, you've been working with Sam", start=11.12, duration=3.84), FetchedTranscriptSnippet(text='Alman for 2 years now. My question to', start=12.4, duration=5.36), FetchedTranscriptSnippet(text='you is what kind of leader is Sam Alman?', start=14.96, duration=5.68), FetchedTranscriptSnippet(text='>> Sam, welcoming the founder and CEO of', start=17.76, duration=3.84), FetchedTranscriptSnippet(text='Open AI.', start=20.64, duration=2.719), FetchedTranscriptSnippet(text='>> We do all of this because we believe the', start=21.6, duration=3.519), FetchedTranscriptSnippet(text='AI is going to be a technological and', start=23.359, duration=3.84), FetchedTranscriptSn

In [10]:
for snippet in transcript_a.snippets:
   print(snippet)

FetchedTranscriptSnippet(text="So what's going to be the future of", start=5.52, duration=4.159)
FetchedTranscriptSnippet(text='software engineers? Are they at threat?', start=6.879, duration=4.241)
FetchedTranscriptSnippet(text='Uh', start=9.679, duration=2.721)
FetchedTranscriptSnippet(text=">> Shimos, you've been working with Sam", start=11.12, duration=3.84)
FetchedTranscriptSnippet(text='Alman for 2 years now. My question to', start=12.4, duration=5.36)
FetchedTranscriptSnippet(text='you is what kind of leader is Sam Alman?', start=14.96, duration=5.68)
FetchedTranscriptSnippet(text='>> Sam, welcoming the founder and CEO of', start=17.76, duration=3.84)
FetchedTranscriptSnippet(text='Open AI.', start=20.64, duration=2.719)
FetchedTranscriptSnippet(text='>> We do all of this because we believe the', start=21.6, duration=3.519)
FetchedTranscriptSnippet(text='AI is going to be a technological and', start=23.359, duration=3.84)
FetchedTranscriptSnippet(text='societal revolution. This 

In [11]:
# indexable
last_snippet = transcript_a[-1].text

# provides a length
snippet_count = len(transcript_a)

print('\'',last_snippet,'\'', 'is the last snippet of a YouTube Video transcript that is', snippet_count, 'words long')

' [Music] ' is the last snippet of a YouTube Video transcript that is 1731 words long


In [12]:
#snippets = transcript_a.snippets          # gives you a list of FetchedTranscriptSnippet
#text = " ".join(s.text for s in snippets) # each snippet has .text
#print(text)

In [13]:
# Flatten it to plain text
transcript = " ".join(chunk.text for chunk in transcript_a.snippets)
print(transcript)

So what's going to be the future of software engineers? Are they at threat? Uh >> Shimos, you've been working with Sam Alman for 2 years now. My question to you is what kind of leader is Sam Alman? >> Sam, welcoming the founder and CEO of Open AI. >> We do all of this because we believe the AI is going to be a technological and societal revolution. This is the best  time ever in the history of technology ever. >> The one and only person who's going to be deciding our futures and sign off. >> So are you telling me that coders have to shift >> train AI systems with thousand times more data? What can you get >> by 2030? What are the possibilities in charge? >> Roll forward 5 years. What are the filters and what are the methods that you would use to filter out the right fit for open AI? >> AI will have so much context as an assistant from me that it is almost going to be >> how does Sam see the future of AI? Sam sees the future in my mind better than anybody else I have worked with. So you

In [14]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.create_documents([transcript])

In [15]:
len(chunks)

75

In [16]:
chunks[70]

Document(metadata={}, page_content="the more the tool can actually take control of you. Until now, we did not have a tool which was intelligent enough. Which is why he speaks about artificial intelligence and artificial consciousness. He says that until now AI is super intelligent but it's not conscious. Do you think we might enter a time where AI could actually develop consciousness? >> Yeah, two questions there. Going back to what you said first, look, yes, AI is is is intelligence. Our history of humanity has been always developing new kinds of tools that are smarter. This is a new level of smartness, but we've always been able to control that, right? And so I believe and that's the optimist in me that believes we can steer this. That's why I'm in this field working on it to make sure we can steer this. Should we think about these risks seriously? Yes, absolutely we should. But I'm extremely optimistic that we can actually steer it to a control consciousness. Um I think we need to d

In [17]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vector_store = FAISS.from_documents(chunks, embeddings)

In [18]:
vector_store.index_to_docstore_id

{0: '76b763b5-44aa-4c17-8487-55a1eab93c9a',
 1: '543e4068-38d9-4c31-946c-1e105b4feec6',
 2: '99a120d9-c961-4cb7-a6f8-4d2a21e57978',
 3: 'ed2b593a-dc0c-445e-906f-61535d904935',
 4: 'c693b9c5-d53b-40e0-917b-12e026eedfc6',
 5: '8c8b2d33-1816-4716-ad20-ecd1deb16c55',
 6: 'e1ac0eba-98d6-4ee6-8e3b-65bd43a5a557',
 7: '3716723b-014e-44e7-ad4c-bc964722d957',
 8: 'c0c1dd93-de8d-4eaf-956a-f62c8d2dea1f',
 9: 'ce390870-8a71-4ca3-9c15-6746d5060f6e',
 10: '6e5da859-4210-4830-b841-724b4c88aac8',
 11: 'dbccacd9-914d-4dcd-a020-9f29577a8d88',
 12: '77e9d958-26bd-4ee9-a537-030fc6a65b81',
 13: '584b6ab6-1ad2-447a-b469-a44581d138d3',
 14: '81040126-e97b-486e-b658-d27659e94a44',
 15: '95ae909c-477b-41fc-98a3-96d0d556cee8',
 16: '8719327f-e05e-4eca-bc58-c9abc82eed09',
 17: '59384641-0cb6-40ca-9509-d78a0cc141b5',
 18: 'cd6ae660-aee0-4692-8500-3804d8241eca',
 19: '23a0e071-2b26-4aab-b9c0-0672a21a6c0b',
 20: '8fbcf63a-e249-44e1-b8b2-509aad7bfc82',
 21: 'b58e00f5-7b34-4f57-8c9d-dfa1d5321582',
 22: '5f21bee9-b3e1-

In [19]:
vector_store.get_by_ids(['71a8cda2-ddee-46e4-82f1-6c8098e8e7d0'])

[]

In [20]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [21]:
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7a28a17ee510>, search_kwargs={'k': 5})

In [22]:
retriever.invoke('What is AI')

[Document(id='82eb70a2-ac0f-4eb5-82b9-148908b11ab8', metadata={}, page_content="to pay their mortgage. What does that mean and how do you imagine the future to be? >> I think there is a a future if AI is helping us accomplish all the tasks with absolute precision. >> Mhm. >> The precision is not there today, right? And this is why you're seeing that like the the simpler tasks require 100% precision and you're not able to do that. Whereas creative tasks have usually more leeway and therefore AI is helping you do that. And uh so uh I think that's that's true. I mean I think the wildest optimal thing that I would love is that it has elevated my ambition a million times. >> I have this incredible AI assistant that deeply understands me. >> It deeply understands all my relationships. It understands what I care about. It understands what I value. And I can I can amplify my productivity. I can amplify my ambition and helps each one of us achieve so much more. >> That is the utopian world that

In [23]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)

In [24]:
prompt = PromptTemplate(
    template="""
      You are a helpful assistant.
      Answer ONLY from the provided transcript context.
      If the context is insufficient, just say you don't know.

      {context}
      Question: {question}
    """,
    input_variables = ['context', 'question']
)

In [25]:
question          = "is the topic of future of AI discussed in this video? if yes then what was discussed"
retrieved_docs    = retriever.invoke(question)

In [26]:
retrieved_docs

[Document(id='76b763b5-44aa-4c17-8487-55a1eab93c9a', metadata={}, page_content="So what's going to be the future of software engineers? Are they at threat? Uh >> Shimos, you've been working with Sam Alman for 2 years now. My question to you is what kind of leader is Sam Alman? >> Sam, welcoming the founder and CEO of Open AI. >> We do all of this because we believe the AI is going to be a technological and societal revolution. This is the best  time ever in the history of technology ever. >> The one and only person who's going to be deciding our futures and sign off. >> So are you telling me that coders have to shift >> train AI systems with thousand times more data? What can you get >> by 2030? What are the possibilities in charge? >> Roll forward 5 years. What are the filters and what are the methods that you would use to filter out the right fit for open AI? >> AI will have so much context as an assistant from me that it is almost going to be >> how does Sam see the future of AI? Sa

In [27]:
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
context_text

"So what's going to be the future of software engineers? Are they at threat? Uh >> Shimos, you've been working with Sam Alman for 2 years now. My question to you is what kind of leader is Sam Alman? >> Sam, welcoming the founder and CEO of Open AI. >> We do all of this because we believe the AI is going to be a technological and societal revolution. This is the best  time ever in the history of technology ever. >> The one and only person who's going to be deciding our futures and sign off. >> So are you telling me that coders have to shift >> train AI systems with thousand times more data? What can you get >> by 2030? What are the possibilities in charge? >> Roll forward 5 years. What are the filters and what are the methods that you would use to filter out the right fit for open AI? >> AI will have so much context as an assistant from me that it is almost going to be >> how does Sam see the future of AI? Sam sees the future in my mind better than anybody else I have worked with. So yo

In [28]:
final_prompt = prompt.invoke({"context": context_text, "question": question})

In [29]:
answer = llm.invoke(final_prompt)
print(answer.content)

Yes, the topic of the future of AI is discussed in the video. It is mentioned that AI is expected to bring about a technological and societal revolution, with the potential to significantly change how tasks are accomplished and improve human lives. The discussion includes the idea that AI will have a deep understanding of societal needs and could guide actions that may not be intuitive to humans. There is also a vision of a future where AI helps achieve precision in tasks, leading to a society where people work for passion rather than necessity. The conversation touches on the potential for AI to create a future similar to major historical revolutions, with advancements in human longevity, relationships, and exploration. Additionally, there is a mention of the importance of addressing risks associated with AI while maintaining optimism about steering its development positively.


In [30]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [31]:
def format_docs(retrieved_docs):
  context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
  return context_text

In [32]:
parallel_chain = RunnableParallel({
    'context': retriever | RunnableLambda(format_docs),
    'question': RunnablePassthrough()
})

In [33]:
parallel_chain.invoke('who is Demis')

{'context': "and even collaborate in real time. So, no more confusion, and no more who's doing what. Just total control. And here's the best part. Your first app with Odo is free for life. That means unlimited projects, full access, and zero cost. So, if you're tired of firefighting, and if you're ready to lead with focus, click the link in the description and start using ODU Project today. Lead smarter and work better with ODU. So today there are several questions that I have and I want to throw light on two very important subjects which I think will be fascinating for the audience. Number one I want to understand how on earth did Sam Ortman build a company as legendary as open AI and number two I want to throw light on the possibilities of AI agents and all of us are just trying to figure out how do we become a beneficiary of the AI revolution and not become a collateral damage. And before we dive deep into it, I want the audience to know who you are. So please introduce yourself and

In [34]:
parser = StrOutputParser()

In [35]:
main_chain = parallel_chain | prompt | llm | parser

In [36]:
main_chain.invoke('Can you summarize the video')

"The video discusses the transformative impact of AI on various industries and the future of software engineering. It emphasizes the optimism surrounding AI, suggesting that it will lead to significant opportunities and innovations across fields like marketing, accounting, legal, and medicine. The conversation touches on the leadership of Sam Altman, the CEO of OpenAI, and his vision for AI's role in society. There is a focus on the potential for AI to create new jobs while also raising concerns about which jobs may become obsolete. The discussion includes the concept of AGI (Artificial General Intelligence) and its economic implications, highlighting that AGI's development is a continuum rather than a binary state. Overall, the message encourages excitement about AI rather than fear, suggesting that individuals and companies should embrace the changes it brings."